In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl

In [ ]:
# image_file = 'C:/Users/tansi/Documents/Imperial_College_London/Year3/FYP_Project/building_train_2_low/sparse/0/images.txt'
# image_file = 'C:/Users/tansi/Documents/Imperial_College_London/Year3/FYP_Project/building_v4_colmap_train/sparse/0/images.txt'
image_file = 'C:/Users/tansi/Documents/Imperial_College_London/Year3/FYP_Project/aligned/images.txt'
true_gt_folder = 'D:/Imperial/FYP/captured_data/airsim_drone_mode/building_train_2/'

In [ ]:
camera_poses = pd.DataFrame()

with open(image_file, "r") as f:
    for line in f.readlines()[4::2]:
        values = line.strip('\n').split(" ")
        name = values[-1]
        q_w, q_x, q_y, q_z = values[1:5]
        r = R.from_quat(np.hstack([float(q_x), float(q_y), float(q_z), float(q_w)]))
        inv_r = r.inv()
        t_x, t_y, t_z = values[5:8]
        t_x, t_y, t_z = float(t_x), float(t_y), float(t_z)
        # x, y, z = np.dot(-(rotmat.T), np.hstack([t_x, t_y, t_z]))
        x, y, z = -inv_r.apply(np.hstack([t_x, t_y, t_z]))
        q_x, q_y, q_z, q_w = inv_r.as_quat()
        camera_poses = camera_poses.append({'ImageFile': name, 
                                            'POS_X': x,
                                            'POS_Y': y,
                                            'POS_Z': z, 
                                            'Q_W': q_w,
                                            'Q_X': q_x,
                                            'Q_Y': q_y,
                                            'Q_Z': q_z}, ignore_index = True)
# camera_poses = pd.DataFrame()

# with open(image_file, "r") as f:
#     for line in f.readlines()[4::2]:
#         values = line.strip('\n').split(" ")
#         name = values[-1]
#         q_w, q_x, q_y, q_z = values[1:5]
#         r = R.from_quat(np.hstack([float(q_y), float(q_x), -float(q_z), float(q_w)]))
#         inv_r = r.inv()
#         t_x, t_y, t_z = values[5:8]
#         t_x, t_y, t_z = float(t), float(t_x), -float(t_z)
#         # x, y, z = np.dot(-(rotmat.T), np.hstack([t_x, t_y, t_z]))
#         x, y, z = -inv_r.apply(np.hstack([t_x, t_y, t_z]))
#         q_x, q_y, q_z, q_w = inv_r.as_quat()
#         camera_poses = camera_poses.append({'ImageFile': name, 
#                                             'POS_X': x,
#                                             'POS_Y': y,
#                                             'POS_Z': z, 
#                                             'Q_W': q_w,
#                                             'Q_X': q_x,
#                                             'Q_Y': q_y,
#                                             'Q_Z': q_z}, ignore_index = True)

In [ ]:
train_data = pd.read_csv(f'{true_gt_folder}airsim_rec.txt', sep="\t")
train_data = train_data[train_data.ImageFile.isin(camera_poses.ImageFile)]
train_data.reset_index(drop=True, inplace=True)

In [ ]:
camera_poses["idx"] = camera_poses.apply(lambda row: int(row["ImageFile"].split(".")[0]), axis = 1)
camera_poses.sort_values(by="idx", inplace= True)
camera_poses.reset_index(drop=True, inplace=True)

In [ ]:
new_y = list(train_data["POS_X"])
new_x = list(train_data["POS_Y"])
new_z = list(- train_data["POS_Z"])
train_data["POS_X"] = new_x
train_data["POS_Y"] = new_y
train_data["POS_Z"] = new_z 

In [ ]:
plt.figure()
ax = plt.axes(projection= '3d')
start = 0
end = -1

ax.plot3D(camera_poses["POS_X"][start:end], camera_poses["POS_Y"][start:end], camera_poses["POS_Z"][start:end], label = "COLMAP Train Poses", color = 'green')
ax.plot3D(train_data["POS_X"][start:end], train_data["POS_Y"][start:end], train_data["POS_Z"][start:end], label = "AirSim Ground Truth Train Poses", color = 'blue')
# ax.set_zlim([-2, -5])
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_zlabel("z (m)")
ax.set_title("AirSim Training Poses and COLMAP Poses")
ax.legend()
plt.show()

In [ ]:
adjusted_colmap_points = pd.DataFrame()
adjusted_colmap_points["POS_X"] = camera_poses["y"]
adjusted_colmap_points["POS_Y"] = camera_poses["x"]
adjusted_colmap_points["POS_Z"] = -camera_poses["z"]
adjusted_colmap_ori = pd.DataFrame()
adjusted_colmap_ori["Q_W"] = camera_poses["q_w"]
adjusted_colmap_ori["Q_X"] = camera_poses["q_x"]
adjusted_colmap_ori["Q_Y"] = camera_poses["q_y"]
adjusted_colmap_ori["Q_Z"] = camera_poses["q_z"]

err =[]
for i in [0, 20, 40, 60]:
    err.append(np.linalg.norm(adjusted_colmap_points[["POS_X", "POS_Y", "POS_Z"]].iloc[i,:] - train_data[["POS_X", "POS_Y", "POS_Z"]].iloc[i,:]))
print(f'Translation error {np.median(err)} (median) m, {np.mean(err)} (mean) m')

q_err =[]
for i in range(len(adjusted_colmap_ori)):
    enu2ned = R.from_quat(np.array([0, 0, math.sqrt(2)/2, math.sqrt(2)/2]))
    ned_ori = enu2ned * R.from_quat(adjusted_colmap_ori[["Q_X", "Q_Y", "Q_Z", "Q_W"]].iloc[i,:])
    q_x, q_y, q_z, q_w = ned_ori.as_quat()
    q_err.append(cal_quat_angle_error(np.array([q_w, q_x, q_y, q_z]), train_data[["Q_W", "Q_X", "Q_Y", "Q_Z"]].iloc[i,:]).item())
print(f'Orientation error {np.median(q_err)} (median) deg, {np.mean(q_err)} (mean) deg')

In [ ]:
def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

In [ ]:
A = '0.png'
B = '100.png'
C = '10000.png'
D = '17000.png'

In [ ]:
a_pose = train_data[train_data.ImageFile == A][['POS_X', 'POS_Y', 'POS_Z']].squeeze()
b_pose = train_data[train_data.ImageFile == B][['POS_X', 'POS_Y', 'POS_Z']].squeeze()
c_pose = train_data[train_data.ImageFile == C][['POS_X', 'POS_Y', 'POS_Z']].squeeze()
d_pose = train_data[train_data.ImageFile == D][['POS_X', 'POS_Y', 'POS_Z']].squeeze()
print(a_pose, b_pose, c_pose, d_pose)

In [ ]:
px = np.sqrt(np.sum((a_pose - c_pose)**2))
a = camera_poses[camera_poses.ImageFile == A][['POS_X','POS_Y','POS_Z']].squeeze()
c = camera_poses[camera_poses.ImageFile == C][['POS_X','POS_Y','POS_Z']].squeeze()
unit_measure = np.sqrt(np.sum((a - c)**2))
print(f'colmap dist {unit_measure}, sim dist {px}')

In [ ]:
colmap_crs = camera_poses[camera_poses.ImageFile.isin([A, B, C, D])][['POS_X','POS_Y','POS_Z']].values.T
cadatastral_crs = train_data[train_data.ImageFile.isin([A, B, C, D])][['POS_X', 'POS_Y', 'POS_Z']].values.T
colmap_crs = colmap_crs / unit_measure * px
# RA + t = B
trans_R, trans_t = rigid_transform_3D(colmap_crs, cadatastral_crs)
print(trans_R, trans_t)

In [ ]:
rotations = camera_poses[['Q_X', 'Q_Y','Q_Z', 'Q_W']].values
transform_rot = R.from_matrix(trans_R)
adjusted_colmap_ori = []
for rot in rotations:
    colmap_rot = R.from_quat(rot)
    trans_quat = (transform_rot * colmap_rot).as_quat()
    if trans_quat[-1] < 0:
        trans_quat = -trans_quat
    adjusted_colmap_ori.append(trans_quat)
adjusted_colmap_ori = pd.DataFrame(adjusted_colmap_ori, columns=["Q_X","Q_Y","Q_Z", "Q_W"])

In [ ]:
plt.figure()
plt.plot(train_data["Q_W"])
plt.plot(adjusted_colmap_ori["Q_W"])
plt.show()

In [ ]:
train_data[['Q_X', 'Q_Y','Q_Z', 'Q_W']].plot()
adjusted_colmap_ori.plot()

In [ ]:
points = camera_poses[['POS_X', 'POS_Y','POS_Z']].values
adjusted_colmap_points = (np.matmul(trans_R, points.T) + trans_t).T
adjusted_colmap_points = pd.DataFrame(adjusted_colmap_points, columns=["POS_X","POS_Y","POS_Z"])

In [ ]:
xyz_gt = train_data.iloc[0,:][['POS_X','POS_Y','POS_Z']].values
xyz_colmap = camera_poses.iloc[0,:][['POS_X','POS_Y','POS_Z']].values
xyz_colmap = xyz_colmap / unit_measure * px
xyz_colmap = (np.matmul(trans_R, xyz_colmap.T) + trans_t.T)

print(xyz_gt)
print(xyz_colmap)

In [ ]:
plt.figure()
plt.plot(train_data["POS_X"], train_data["POS_Z"], marker=".")
plt.plot(adjusted_colmap_points["POS_X"], adjusted_colmap_points["POS_Z"], marker=".")
plt.show()

In [ ]:
plt.figure()
ax = plt.axes(projection= '3d')
start = 0
end = 200

ax.plot3D(adjusted_colmap_points["POS_X"][start:end], adjusted_colmap_points["POS_Y"][start:end], adjusted_colmap_points["POS_Z"][start:end], label = "COLMAP Train Poses", color = 'green')
ax.plot3D(train_data["POS_X"][start:end], train_data["POS_Y"][start:end], train_data["POS_Z"][start:end], label = "AirSim Ground Truth Train Poses", color = 'blue')
# ax.set_zlim([-2, -5])
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_zlabel("z (m)")
ax.set_title("AirSim Training Poses and COLMAP Poses")
ax.legend()
plt.show()

In [ ]:
def cal_quat_angle_error(label, pred):
    if len(label.shape) == 1:
        label = np.expand_dims(label, axis=0)
    if len(pred.shape) == 1:
        pred = np.expand_dims(pred, axis=0)
    q1 = pred / np.linalg.norm(pred, axis=1, keepdims=True)
    q2 = label / np.linalg.norm(label, axis=1, keepdims=True)
    d = np.abs(np.sum(np.multiply(q1,q2), axis=1, keepdims=True)) # Here we have abs()

    d = np.clip(d, a_min=-1, a_max=1)
    error = 2 * np.degrees(np.arccos(d))
    return error

In [ ]:
err =[]
for i in range(len(adjusted_colmap_ori)):
    err.append(np.linalg.norm(adjusted_colmap_points[["POS_X", "POS_Y", "POS_Z"]].iloc[i,:] - train_data[["POS_X", "POS_Y", "POS_Z"]].iloc[i,:]))
print(f'Translation error {np.median(err)} (median) m, {np.mean(err)} (mean) m')

q_err =[]
for i in range(len(adjusted_colmap_ori)):
    q_err.append(cal_quat_angle_error(adjusted_colmap_ori[["Q_W", "Q_X", "Q_Y", "Q_Z"]].iloc[i,:], train_data[["Q_W", "Q_X", "Q_Y", "Q_Z"]].iloc[i,:]).item())
print(f'Orientation error {np.median(q_err)} (median) deg, {np.mean(q_err)} (mean) deg')

In [ ]:
q_err =[]
enus = []
ned2enu = R.from_quat(np.array([-math.sqrt(2)/2, -math.sqrt(2)/2, 0, 0]))
for i in range(len(camera_poses)):
    ned = R.from_quat(np.array(train_data[["Q_X", "Q_Y", "Q_Z", "Q_W"]].iloc[i,:]))
    enu = (ned.inv() * ned2enu).inv().as_quat()
    enus.append(enu)
    q_err.append(cal_quat_angle_error(camera_poses[["Q_X", "Q_Y", "Q_Z", "Q_W"]].iloc[i,:], enu).item())
print(f'Orientation error {np.median(q_err)} (median) deg, {np.mean(q_err)} (mean) deg')

In [ ]:
enus = pd.DataFrame(enus, columns=["Q_X","Q_Y","Q_Z", "Q_W"])
enus.plot()

In [ ]:
train_data_mod = train_data.copy()
train_data_mod["POS_X"] = adjusted_colmap_points["POS_X"]
train_data_mod["POS_Y"] = adjusted_colmap_points["POS_Y"]
train_data_mod["POS_Z"] = adjusted_colmap_points["POS_Z"]
train_data_mod["Q_X"] = adjusted_colmap_ori["Q_X"]
train_data_mod["Q_Y"] = adjusted_colmap_ori["Q_Y"]
train_data_mod["Q_Z"] = adjusted_colmap_ori["Q_Z"]
train_data_mod["Q_W"] = adjusted_colmap_ori["Q_W"]

In [ ]:
train_data_mod.to_csv(f'{true_gt_folder}/colmap_v2.txt', header=True, index=None, sep='\t', mode='w')

In [ ]:
train_data_mod[["Q_W", "Q_X", "Q_Y", "Q_Z"]].plot()